World     : Grid 

Algorithm : N-step SARSA

Sampling  : On-Policy

Policy    : Softmax

Paramterization : Neural Network

In [ ]:
import numpy as np
import torch
import random

In [ ]:

# Reward -1
# we can move through potholes but we would take Reward -5 for time wasted
# actions = ("up","right","down","left") 
# state = ("N","NE","E","SE","S","SW","W","NW", "self_x", "self_y")

In [ ]:
def score_board(state):
    
    print(state[7], state[0], state[1])
    print(state[6], world[int(state[8])][int(state[9])], state[2])
    print(state[5], state[4], state[3])

    print("\nPos :",state[8],state[9])

In [ ]:
def gen_world(length,t_x,t_y,a_x,a_y):
    choices = [1,1,0,0,0]

    world = np.random.choice(choices, size=(length,length))

    #agent position
    #target position

    world[t_x][t_y] = -1
    world[a_x][a_y] = 0

    return world
# 1 are potholes walls, any cell outside the world is inaccesible an throws you back with -2 reward

In [ ]:
# knows every cell around it, even diagonal ones, and its own position

def env(action,state):
    new_state = state.clone().detach()
    
    
    a_x = int(new_state[8]*length -1)
    a_y = int(new_state[9]*length -1)
    # print(a_x,a_y)
    reward = torch.tensor(-1)
    
    if action == "u":
        if a_x == 0:
            reward -= 4
            e_t = state/2
            e_t[8] = 0
            e_t[9] = 0 
            e_t += new_state
            return reward, new_state,e_t, a_x,a_y
        a_x += -1

    elif action == "d":
        if a_x == length-1:
            reward -= 4
            e_t = state/2
            e_t[8] = 0
            e_t[9] = 0 
            e_t += new_state
            return reward, new_state,e_t, a_x,a_y
        a_x += 1
    
    elif action == "r":
        if a_y == length-1:
            reward -= 4
            e_t = state/2
            e_t[8] = 0
            e_t[9] = 0 
            e_t += new_state
            return reward, new_state,e_t, a_x,a_y
        a_y += 1
    
    elif action == "l":
        if a_y == 0:
            reward -= 4
            e_t = state/2
            e_t[8] = 0
            e_t[9] = 0 
            e_t += new_state
            return reward, new_state,e_t, a_x,a_y
        a_y += -1
    
    new_state[8] = (a_x+1)/length
    new_state[9] = (a_y+1)/length
    
    if a_x>0:
        new_state[0] = world[a_x-1][a_y]
    else:
        new_state[0] = 1

    if a_x>0 and a_y<length-1:
        new_state[1] = world[a_x-1][a_y+1]
    else:
        new_state[1] = 1

    if a_y<length-1:
        new_state[2] = world[a_x][a_y+1]
    else:
        new_state[2] = 1
    
    if a_x<length-1 and a_y<length-1:
        new_state[3] = world[a_x+1][a_y+1]
    else:
        new_state[3] = 1
    
    if a_x<length-1:
        new_state[4] = world[a_x+1][a_y]
    else:
        new_state[4] = 1

    if a_x<length-1 and a_y>0:
        new_state[5] = world[a_x+1][a_y-1]
    else:
        new_state[5] = 1

    if a_y>0:    
        new_state[6] = world[a_x][a_y-1]
    else:
        new_state[6] = 1

    if a_x>0 and a_y>0:
        new_state[7] = world[a_x-1][a_y-1]
    else:
        new_state[7] = 1


    if world[a_x][a_y] == 1:
        reward -= 4

    elif world[a_x][a_y]== -1:
        reward += 10
    
    
    e_t = state/2
    e_t[8] = 0
    e_t[9] = 0 
    e_t += new_state

    return reward, new_state,e_t, a_x,a_y

In [ ]:
# simple 1 hidden layer NN

class FApproxModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(10, 40)
        self.linear2 = torch.nn.Linear(40, 20)
        self.linear3 = torch.nn.Linear(20, 10)
        self.linear4 = torch.nn.Linear(10, 4)
    
    def forward(self,features):
        
        out = torch.relu(self.linear1(features))
        out = torch.relu(self.linear2(out))
        out = self.linear3(out)        
        action_pref = self.linear4(out)
        
        return action_pref

Q = FApproxModel()

In [ ]:
def softmax_policy(action_pref, actions=("u","r","d","l") ):

    # print(action_pref)
    action_prob = torch.softmax(torch.relu(action_pref), -1)
    # print(action_prob)
    
    choice = random.choices((0,1,2,3),action_prob)
    
    return actions[choice[0]],action_pref[choice[0]],action_prob[choice[0]]


def e_greedy_policy(h_f,a_x,a_y, actions=("u","r","d","l")):
    greedy = random.choices((0,1),(0.30,0.70))[0]
    
    if greedy:
        choice = torch.where(h_f==h_f.max())[0][0]
        
    else:
        if a_x<15 and a_y<15:
            if a_x<a_y:
                choice = 2
            else: 
                choice = 1
        else:
            return softmax_policy(h_f)
            
    return actions[choice],h_f[choice],0
    



In [ ]:
def sample(state_temp,e_t,n,r,gamma=0.9):
    q = int(r)
    discount = gamma
    
    for i in range(1,n):
        action_pref = Q(e_t)
        a,q_T,p_t = e_greedy_policy(action_pref,a_x,a_y)

        reward, state_temp,e_t,a_x,a_y = env(a, state_temp)
        
        q += discount*reward
        discount *= gamma

    action_pref = Q(state_temp)
    with torch.no_grad():
        q += discount*action_pref.max()

    return q

In [ ]:
# def loss_fn(Q_pi,Q_t):

#     error = Q_pi - Q_t
#     loss = 0.5*(error**2)

#     return loss

In [ ]:
opt = torch.optim.SGD(Q.parameters(), lr=0.0001)
loss_fn = torch.nn.MSELoss()

In [ ]:
net_episode = 0
history = []

length = 20
    

In [ ]:
episodes = 100


for episode in range(episodes):
    with torch.no_grad():
        
        net_episode+=1

        G = torch.tensor(0)
        a_x, a_y = 0,0
        t_x, t_y = np.random.randint(length/2+length/5,length), np.random.randint(length/2+length/5,length)
        world = torch.tensor(gen_world(20, t_x,t_y, a_x,a_y),dtype=torch.int8)
        path = torch.zeros((length,length),dtype=torch.int8)
        state = torch.Tensor([1,1,world[0][1],world[1][1],world[1][0],1,1,1,(a_x+1)/length,(a_y+1)/length])

        path[a_x][a_y] = 1
        t = 1
    opt.zero_grad()
    while world[a_x][a_y] != -1 and t<5000:
        
        h_f = Q(state)
        a,q_t,p_t = e_greedy_policy(h_f,a_x,a_y)
        # print(action_pref)
        # print(a)
        
        reward, state,e_t, a_x,a_y = env(a, state)

        # n-step SARSA 
        # q_pi = sample(state,e_t, 5,reward, a_x,a_y)
        
        with torch.no_grad():
            max_q = h_f.max()
        q_pi = reward + 0.9*max_q

        loss = loss_fn(q_pi, q_t)
        loss.backward()

        opt.step()
        opt.zero_grad()  

        with torch.no_grad():
            G += reward

            if path[a_x][a_y] + 1 <10:
                path[a_x][a_y] += 1
            
            t += 1
            # if t%1000 == 0:
            #     print(".",end="")
    
    with torch.no_grad():
        print("Episode :", net_episode, " Score :", int(G), " Time :",t)

        if G>-90:
            print(G, t, G/t)
            print(path)
            
            torch.save(Q.state_dict(),f"{G}|{t}.pth")

        # print(G/t)
        # print(path)

        # print("Distance",d,"%")
        # history.append((G,t,G/t))